# Modelling herb runs in Old School Runescape

## 03-Data Collection & Prior Modelling
As the only way to interface with Old School Runescapes Farming skill is through playing the game, my method of data collection will be to record data while doing herb runs myself. I will use a .csv formatted by the rows denoting the herb run's number and the columns denoting the herb patches location. After a successful harvest at a particular patch I will enter the exact number of herbs picked into the corresponding entry by herb patch location and herb run number. Note that the number I enter in each entry will also include the minimum possible amount of herbs harvested based on the compost I used on the herb patch. In the case of ultra-compost I am guaranteed to pick 6 herbs. So in order to make correct statistical inference on the probability in question I will need to subtract 6 from each entry when doing analysis. In order to lock down as many parameters related to the calculation of the first failure to harvest an extra herb from a patch, I will plant the same type of herb in each of the patches. The herb I will plant is the highest tier herb available, torstol. As for the other dependencies of the probability in question I will remain at 85 farming over the data collection period, I will always treat each herb patch with ultra-compost and I will always be using magic secateurs while harvesting. This ensures that each herb run will be an independent sample as the other known parameters will remain fixed over each harvest between particular patches. Also as each individual patch will have a different probability of first failure to harvest a herb we cannot combine the data for each herb patch, thus, the need to seperate the data by herb patch location. So we are going to be computing eight instances of this probability, one for each herb patch. Once we have these eight individual probabilities we can then simulate a full herb run by adding realisations of harvesting of each herb patch together. 

The eight herb patches will be labelled by their in-game location as: Farming Guild, Port Phasmatys, Weiss, Catherby, Troll Stronghold, Hosidius, Ardougne and Falador. So the probability distribution that we will be using MLE to estmate for each patch will be (where $X_i$ for $i$ being one of the above names, will be the total amount of herbs harvested on a single herb patch and $p_i$ will be the true probability of failing to pick an extra herb from a single herb patch)
$$P(X_i = x_i) = 6 + p_i^{x_i}(1-p_i), $$

where MLE gave our estimate of $p_i$ to be
$$\text{MLE}(p_i) = \frac{\bar{X_i}}{1+\bar{X_i}}.$$

This gives our probability distribution of total herbs harvested on a single herb run to be (where $T$  is this total amount)
$$P(T = t) = \sum_{\forall i}[6 + p_i^x(1-p_i)] = 40 + \sum_{\forall i} p_i^{x_i}(1-p_i).$$
